In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.inspection import permutation_importance
from sklearn.model_selection import RandomizedSearchCV
data = pd.read_csv('test.csv')

In [2]:
data.duplicated().sum()
data.drop_duplicates(inplace=True)
data.shape
data

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,5000002,Male,44,1,0.0,1,1-2 Year,No,2630.0,122.0,107,0
1,5000003,Male,25,1,28.0,0,< 1 Year,No,2630.0,152.0,248,0
2,5000004,Female,26,1,2.0,1,< 1 Year,No,38414.0,160.0,24,0
3,5000005,Male,28,1,8.0,1,< 1 Year,No,44203.0,152.0,185,0
4,5000006,Female,59,1,8.0,0,1-2 Year,Yes,2630.0,155.0,239,0
...,...,...,...,...,...,...,...,...,...,...,...,...
6504791,11504793,Male,48,1,6.0,0,1-2 Year,Yes,27412.0,26.0,218,0
6504792,11504794,Female,26,1,36.0,0,< 1 Year,Yes,29509.0,152.0,115,1
6504793,11504795,Female,29,1,32.0,1,< 1 Year,No,2630.0,152.0,189,0
6504794,11504796,Female,51,1,28.0,0,1-2 Year,Yes,48443.0,26.0,274,1


In [3]:
categorical_features = ['Gender', 'Vehicle_Damage']
data = pd.get_dummies(data, columns=categorical_features, drop_first=True)
data_o = data
data_o = pd.get_dummies(data, columns=['Vehicle_Age'], prefix='Vehicle_Age')
data_o

,id,Age,Driving_License,Region_Code,Previously_Insured,Annual_Premium,Policy_Sales_Channel,Vintage,Response,Gender_Male,Vehicle_Damage_Yes,Vehicle_Age_1-2 Year,Vehicle_Age_< 1 Year,Vehicle_Age_> 2 Years
0,5000002,44,1,0.0,1,2630.0,122.0,107,0,True,False,True,False,False
1,5000003,25,1,28.0,0,2630.0,152.0,248,0,True,False,False,True,False
2,5000004,26,1,2.0,1,38414.0,160.0,24,0,False,False,False,True,False
3,5000005,28,1,8.0,1,44203.0,152.0,185,0,True,False,False,True,False
4,5000006,59,1,8.0,0,2630.0,155.0,239,0,False,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6504791,11504793,48,1,6.0,0,27412.0,26.0,218,0,True,True,True,False,False
6504792,11504794,26,1,36.0,0,29509.0,152.0,115,1,False,True,False,True,False
6504793,11504795,29,1,32.0,1,2630.0,152.0,189,0,False,False,False,True,False
6504794,11504796,51,1,28.0,0,48443.0,26.0,274,1,False,True,True,False,False


In [4]:
# 1. Словарь для переименования (API не любит пробелы и знаки < >)
rename_map = {
    'Vehicle_Age_< 1 Year': 'Vehicle_Age_lt_1_Year',
    'Vehicle_Age_> 2 Years': 'Vehicle_Age_gt_2_Years',
    'Vehicle_Age_1-2 Year': 'Vehicle_Age_1_2_Year'
}

# 2. Переименовываем колонки в копии датафрейма
df_clean = data_o.rename(columns=rename_map).copy()

# 3. Список колонок, которые ждет наш API скрипт
cols_to_keep = [
    'Age',
    'Driving_License',
    'Previously_Insured',
    'Annual_Premium',
    'Gender_Male',
    'Vehicle_Damage_Yes',
    'Vehicle_Age_lt_1_Year',
    'Vehicle_Age_1_2_Year',
    'Vehicle_Age_gt_2_Years',
    'Response'
]

# 4. Оставляем только нужное
df_clean = df_clean[cols_to_keep]


In [5]:

# Сохранить в CSV
df_clean.to_csv('data.csv', index=False, encoding='utf-8')

print("Данные сохранены в data.csv")

Данные сохранены в data.csv


In [ ]:
import pandas as pd
import pickle
import time
from typing import List, Tuple
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# ==================== КОНФИГУРАЦИЯ ====================
MODEL_FILE = "lgb_model.pkl"  # Путь к вашей модели
CSV_FILE = "data.csv"
# ==================== ====================

class LocalModelProcessor:
    def __init__(self, model_file: str):
        print(f"Загружаю модель из {model_file}...")
        try:
            with open(model_file, 'rb') as f:
                self.model = pickle.load(f)
            print("✅ Модель загружена успешно\n")
        except FileNotFoundError:
            raise FileNotFoundError(f"Файл модели '{model_file}' не найден! Проверьте путь.")

        self.y_true = []
        self.y_pred = []
        self.total_processed = 0

    def build_features(self, row: pd.Series, cols: List[str]) -> List[float]:
        """Преобразуем строку в вектор признаков"""
        features = []
        for col in cols:
            try:
                val = row[col]
                # Приводим типы
                if col in ["Age", "Annual_Premium"]:
                    features.append(float(val))
                elif col in ["Driving_License", "Previously_Insured",
                             "Gender_Male", "Vehicle_Damage_Yes",
                             "Vehicle_Age_1_2_Year", "Vehicle_Age_lt_1_Year",
                             "Vehicle_Age_gt_2_Years"]:
                    features.append(float(int(val)))
                else:
                    features.append(float(val))
            except (KeyError, ValueError, TypeError):
                # Если что-то пошло не так, ставим 0.0 (безопасный fallback)
                features.append(0.0)
        return features

    def process_csv(self, csv_file: str) -> Tuple[List[int], List[int], float]:
        """Процесс данные из CSV с прямой обработкой моделью"""
        print(f"Читаю данные из {csv_file}...")
        start = time.time()

        # Читаем весь файл сразу (pandas делает это очень быстро)
        try:
            df = pd.read_csv(csv_file)
        except FileNotFoundError:
             raise FileNotFoundError(f"Файл данных '{csv_file}' не найден!")

        print(f"Загружено строк: {len(df):,}")

        if "Response" not in df.columns:
            raise ValueError("ОШИБКА: В CSV должна быть колонка 'Response'!")

        # Определяем колонки признаков (все кроме целевой)
        all_columns = list(df.columns)
        feature_columns = [c for c in all_columns if c != "Response"]
        print(f"Признаки для модели ({len(feature_columns)}): {feature_columns}\n")

        print("Преобразуем данные в формат модели...")
        all_features = []
        true_labels = []

        # Проходим по датафрейму
        # itertuples() быстрее, чем iterrows()
        # Но для надежности используем ручной сбор, чтобы гарантировать порядок колонок
        for idx, row in df.iterrows():
            try:
                # Получаем истинный лейбл
                target = row.get("Response")
                if pd.isna(target):
                    continue
                true_label = int(target)

                # Собираем признаки
                features = self.build_features(row, feature_columns)

                all_features.append(features)
                true_labels.append(true_label)
                self.total_processed += 1

                if self.total_processed % 500000 == 0:
                    print(f"  Подготовлено: {self.total_processed:,} строк...")
            except Exception:
                pass

        print(f"Данные готовы: {len(all_features):,} строк.")

        if not all_features:
            print("❌ Нет данных для обработки!")
            return [], [], 0.0

        # Предсказываем ВСЕ сразу (Batch Inference)
        print("🚀 Запускаю предсказание модели...")
        predictions = self.model.predict(all_features)

        self.y_true = true_labels
        self.y_pred = predictions.tolist()

        elapsed = time.time() - start
        return self.y_true, self.y_pred, elapsed


def main():
    print("=" * 70)
    print("СКОРОСТНАЯ ОЦЕНКА МОДЕЛИ (LOCAL INFERENCE)")
    print("=" * 70)

    try:
        processor = LocalModelProcessor(MODEL_FILE)
        y_true, y_pred, elapsed = processor.process_csv(CSV_FILE)

        print("\n" + "=" * 70)
        print("ИТОГИ ОБРАБОТКИ")
        print("=" * 70)
        total = len(y_true)

        if total == 0:
            print("Нет данных для анализа.")
            return

        print(f"Всего обработано:      {total:,}")
        print(f"Время выполнения:      {elapsed:.2f} сек ({elapsed/60:.2f} мин)")
        if elapsed > 0:
            print(f"Скорость обработки:    {total/elapsed:,.0f} строк/сек")

        print("\n" + "=" * 70)
        print("📊 МЕТРИКИ КАЧЕСТВА МОДЕЛИ")
        print("=" * 70)

        # Расчет метрик
        acc = accuracy_score(y_true, y_pred)
        prec = precision_score(y_true, y_pred, zero_division=0)
        rec = recall_score(y_true, y_pred, zero_division=0)
        f1 = f1_score(y_true, y_pred, zero_division=0)

        # Матрица ошибок
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

        # Вывод основных метрик
        print(f"Accuracy (Точность):   {acc:.4f}")
        print(f"Precision (Точность+): {prec:.4f}")
        print(f"Recall (Полнота):      {rec:.4f}")
        print(f"F1-Score:              {f1:.4f}")

        print("-" * 70)
        print("МАТРИЦА ОШИБОК (CONFUSION MATRIX):")
        print("-" * 70)
        print(f"✅ True Positives (TP) - Верно предсказано 'Да':   {tp:,}")
        print(f"✅ True Negatives (TN) - Верно предсказано 'Нет':  {tn:,}")
        print(f"❌ False Positives (FP) - Ложная тревога:          {fp:,}")
        print(f"❌ False Negatives (FN) - Пропуск цели:            {fn:,}")
        print("-" * 70)
        print(f"Всего положительных (TP+FN): {tp+fn:,}")
        print(f"Всего отрицательных (TN+FP): {tn+fp:,}")
        print("=" * 70)

    except Exception as e:
        print(f"\n❌ ПРОИЗОШЛА ОШИБКА: {e}")

if __name__ == "__main__":
    main()


СКОРОСТНАЯ ОЦЕНКА МОДЕЛИ (LOCAL INFERENCE)
Загружаю модель из lgb_model.pkl...
✅ Модель загружена успешно

Читаю данные из data.csv...


C:\Users\IVAN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.7.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\IVAN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator RandomizedSearchCV from version 1.7.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Загружено строк: 6,504,796
Признаки для модели (9): ['Age', 'Driving_License', 'Previously_Insured', 'Annual_Premium', 'Gender_Male', 'Vehicle_Damage_Yes', 'Vehicle_Age_lt_1_Year', 'Vehicle_Age_1_2_Year', 'Vehicle_Age_gt_2_Years']

Преобразуем данные в формат модели...
  Подготовлено: 500,000 строк...
  Подготовлено: 1,000,000 строк...
  Подготовлено: 1,500,000 строк...
  Подготовлено: 2,000,000 строк...
  Подготовлено: 2,500,000 строк...
  Подготовлено: 3,000,000 строк...
  Подготовлено: 3,500,000 строк...
  Подготовлено: 4,000,000 строк...
  Подготовлено: 4,500,000 строк...
  Подготовлено: 5,000,000 строк...
  Подготовлено: 5,500,000 строк...
  Подготовлено: 6,000,000 строк...
  Подготовлено: 6,500,000 строк...
Данные готовы: 6,504,796 строк.
🚀 Запускаю предсказание модели...

ИТОГИ ОБРАБОТКИ
Всего обработано:      6,504,796
Время выполнения:      171.69 сек (2.86 мин)
Скорость обработки:    37,887 строк/сек

📊 МЕТРИКИ КАЧЕСТВА МОДЕЛИ
Accuracy (Точность):   0.7901
Precision (Точность